In [1]:
import ipl 

organism = ipl.Organism()
organism.verbosity = 1

game = ipl.games.ElMazeGame(3,2)
organism.configure(game.player_config())

action_program = [
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0]
]


In [2]:
organism.reset_state()
for iturn, pa in enumerate(action_program):
  print('\nRUNNER: Turn {}'.format(iturn + 1))
  organism.handle_sensor_input(game.sensors())

  oa = organism.choose_action(pa)
  if oa.outcomes and len(oa.outcomes):
    print('Expected outcomes:')
    for outcome in oa.outcomes:
      print('\t{}'.format(outcome))
  else:
    print('(Action has no outcomes)')

  game.act(oa.actuators)
  

print()
print('RUNNER: Predefined action sequence complete.')
organism.handle_sensor_input(game.sensors())




RUNNER: Turn 1
ORGANISM: Received sensor input: [1, 0, 0, 0, 0]
ORGANISM: Committing to action: ACTION: [1, 0, 0, 0] (-> 9 outcomes)
Expected outcomes:
	OUTCOME: [1, 1, 0, 1, 0] (11% $0.00 = $0.00)
	OUTCOME: [0, 0, 0, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [0, 1, 1, 0, 0] (11% $0.00 = $0.00)
	OUTCOME: [1, 1, 1, 0, 0] (11% $0.00 = $0.00)
	OUTCOME: [1, 0, 0, 0, 0] (11% $0.00 = $0.00)
	OUTCOME: [1, 0, 1, 1, 0] (11% $0.00 = $0.00)
	OUTCOME: [0, 1, 1, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 1, 1, 0, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 1, 1, 1, 1] (11% $1.00 = $0.11)

RUNNER: Turn 2
ORGANISM: Received sensor input: [1, 0, 0, 1, 0]
ORGANISM: Experience repo size: 1
ORGANISM: Committing to action: ACTION: [1, 0, 0, 0] (-> 9 outcomes)
Expected outcomes:
	OUTCOME: [1, 0, 0, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [0, 0, 0, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [0, 0, 0, 0, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 1, 1, 0, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 1, 1, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [0, 0,

In [3]:
game.io_vector_labels()

{'sensors': ['FORWARD', 'LEFT', 'RIGHT', 'BACK', 'VICTORY'],
 'actuators': ['GO', 'TURN LEFT', 'TURN RIGHT', 'TURN BACK']}

In [7]:
print('# learned vectors: {}'.format(len(organism.experience_repo.experiences)))
for i in range(0,31):
    a = [int(x) for x in list('{:05b}'.format(i))]
    est = organism.outcome_likelihood_estimator.estimate( ipl.nnplanner.Experience([1, 0, 0, 1, 0], [1, 0, 0, 0],  a))
    print('{}: {:2d}%'.format(a, int(est*100)))


# learned vectors: 66
[0, 0, 0, 0, 0]:  4%
[0, 0, 0, 0, 1]: 54%
[0, 0, 0, 1, 0]:  9%
[0, 0, 0, 1, 1]:  2%
[0, 0, 1, 0, 0]: 11%
[0, 0, 1, 0, 1]: 14%
[0, 0, 1, 1, 0]: 11%
[0, 0, 1, 1, 1]: -5%
[0, 1, 0, 0, 0]: 11%
[0, 1, 0, 0, 1]:  0%
[0, 1, 0, 1, 0]: 43%
[0, 1, 0, 1, 1]: 12%
[0, 1, 1, 0, 0]: -4%
[0, 1, 1, 0, 1]: -5%
[0, 1, 1, 1, 0]:  6%
[0, 1, 1, 1, 1]: -2%
[1, 0, 0, 0, 0]: -3%
[1, 0, 0, 0, 1]:  4%
[1, 0, 0, 1, 0]: 61%
[1, 0, 0, 1, 1]:  2%
[1, 0, 1, 0, 0]: -3%
[1, 0, 1, 0, 1]: -3%
[1, 0, 1, 1, 0]:  1%
[1, 0, 1, 1, 1]: -12%
[1, 1, 0, 0, 0]:  8%
[1, 1, 0, 0, 1]: -6%
[1, 1, 0, 1, 0]: 19%
[1, 1, 0, 1, 1]:  0%
[1, 1, 1, 0, 0]: -6%
[1, 1, 1, 0, 1]:  6%
[1, 1, 1, 1, 0]:  0%


In [5]:
organism.verbosity = 0
for i in range(0, 10):
    print('Run #{}'.format(i))
    game = ipl.games.ElMazeGame(3,2)    

    organism.reset_state()
    for pa in action_program:
      organism.handle_sensor_input(game.sensors())
      oa = organism.choose_action(pa)
      game.act(oa.actuators)
    organism.handle_sensor_input(game.sensors())
    

Run #0
Run #1
Run #2
Run #3
Run #4
Run #5
Run #6
Run #7
Run #8
Run #9


In [6]:
import sklearn

# nn = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(20))
nn = sklearn.neural_network.MLPRegressor(
        hidden_layer_sizes=(32),
        activation='logistic',
        solver='sgd',
        warm_start=True)


Xall = [v[0] for v in organism.outcome_likelihood_estimator.all_learned]
yall = [v[1] for v in organism.outcome_likelihood_estimator.all_learned]

#nn.fit(Xall, yall)
for x, y in zip(Xall, yall):
    nn.partial_fit([x], [y])

AttributeError: 'OutcomeLikelihoodEstimator' object has no attribute 'all_learned'

In [ ]:
for i in range(0,31):
    a = [int(x) for x in list('{:05b}'.format(i))]
    est = nn.predict([[0, 1, 0, 1, 0] + [0, 1, 0, 0] + a])
    print('{}: {:2d}%'.format(a, int(est*100)))


In [ ]:
organism.outcome_likelihood_estimator.neuralnet